# SVM

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder


### Prepering data

In [31]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [32]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors="coerce")

In [33]:
df.dropna(inplace=True)

In [34]:
df_d = df.drop("customerID", axis = 1)

In [35]:
df_d['Churn'].replace(to_replace='Yes', value=1, inplace=True)
df_d['Churn'].replace(to_replace='No',  value=0, inplace=True)

#### Skalowanie

In [36]:
df_d.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [37]:
def apply_scalers(df, columns_to_exclude=None):
    if columns_to_exclude:
        exclude_filter = ~df.columns.isin(columns_to_exclude) 
    else:
        exclude_filter = ~df.columns.isin([]) 
    for column in df.iloc[:, exclude_filter].columns:
        df[column] = df[column].astype(float)
 
    df.loc[:, exclude_filter] = StandardScaler().fit_transform(df.loc[:, exclude_filter])
    return df
 
df_d = apply_scalers(df_d, columns_to_exclude=['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn'])

#### Dumifikacja

In [38]:
dummies_df = pd.get_dummies(df_d)

In [39]:
dummies_df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,-1.280248,-1.161694,-0.994194,0,1,0,0,1,1,...,0,1,0,0,0,1,0,0,1,0
1,0,0.064303,-0.260878,-0.173740,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,1
2,0,-1.239504,-0.363923,-0.959649,1,0,1,1,0,1,...,0,1,0,0,0,1,0,0,0,1
3,0,0.512486,-0.747850,-0.195248,0,0,1,1,0,1,...,0,0,1,0,1,0,1,0,0,0
4,0,-1.239504,0.196178,-0.940457,1,1,0,1,0,1,...,0,1,0,0,0,1,0,0,1,0


### Train test split

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
y = dummies_df['Churn'].values
X = dummies_df.drop(columns = ['Churn'])

#### Train test split NEW

In [42]:
#Splity X,y into train, val, test
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state=44,train_size=0.6,stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, random_state=44,train_size=0.5,stratify=y_val)

### Clasification witg Grid Search CV for SVM model

In [43]:
from sklearn import svm, metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
import matplotlib.pyplot as plt

In [44]:
parameters = {"kernel":("linear", "rbf"), "C":[1,2,3]}

In [45]:
svm_clsf = svm.SVC()
grid_clsf = GridSearchCV(estimator=svm_clsf, param_grid=parameters ,n_jobs=1, verbose=2, scoring="f1")

In [46]:
grid_clsf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END .................................C=1, kernel=linear; total time=   1.0s
[CV] END .................................C=1, kernel=linear; total time=   0.9s
[CV] END .................................C=1, kernel=linear; total time=   0.9s
[CV] END .................................C=1, kernel=linear; total time=   1.0s
[CV] END .................................C=1, kernel=linear; total time=   1.0s
[CV] END ....................................C=1, kernel=rbf; total time=   1.2s
[CV] END ....................................C=1, kernel=rbf; total time=   1.3s
[CV] END ....................................C=1, kernel=rbf; total time=   1.3s
[CV] END ....................................C=1, kernel=rbf; total time=   1.2s
[CV] END ....................................C=1, kernel=rbf; total time=   1.0s
[CV] END .................................C=2, kernel=linear; total time=   0.9s
[CV] END .................................C=2, ke

GridSearchCV(estimator=SVC(), n_jobs=1,
             param_grid={'C': [1, 2, 3], 'kernel': ('linear', 'rbf')},
             scoring='f1', verbose=2)

In [47]:
grid_clsf.cv_results_

{'mean_fit_time': array([0.94002624, 0.90687885, 0.96398616, 0.80458755, 1.10703526,
        0.82307544]),
 'std_fit_time': array([0.0367982 , 0.08510609, 0.02887175, 0.02156775, 0.02308065,
        0.01703524]),
 'mean_score_time': array([0.11847792, 0.36318483, 0.10165801, 0.33761797, 0.10028639,
        0.32696466]),
 'std_score_time': array([0.0031689 , 0.0360987 , 0.00519002, 0.04496073, 0.00245228,
        0.00986092]),
 'param_C': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 2, 'kernel': 'linear'},
  {'C': 2, 'kernel': 'rbf'},
  {'C': 3, 'kernel': 'linear'},
  {'C': 3, 'kernel': 'rbf'}],
 'split0

### Clasification with random search for SVM model

In [48]:
svm_clsf = svm.SVC()
random_clsf = RandomizedSearchCV(estimator=svm_clsf, param_distributions=parameters, n_iter=1, cv=3, n_jobs=1, verbose=2, scoring="f1")

In [49]:
random_clsf.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END .................................C=2, kernel=linear; total time=   0.7s
[CV] END .................................C=2, kernel=linear; total time=   0.7s
[CV] END .................................C=2, kernel=linear; total time=   0.7s


RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=1, n_jobs=1,
                   param_distributions={'C': [1, 2, 3],
                                        'kernel': ('linear', 'rbf')},
                   scoring='f1', verbose=2)

In [50]:
random_clsf.cv_results_

{'mean_fit_time': array([0.65756361]),
 'std_fit_time': array([0.02639649]),
 'mean_score_time': array([0.12906122]),
 'std_score_time': array([0.00622358]),
 'param_kernel': masked_array(data=['linear'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[2],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'linear', 'C': 2}],
 'split0_test_score': array([0.58611111]),
 'split1_test_score': array([0.55472264]),
 'split2_test_score': array([0.57522124]),
 'mean_test_score': array([0.57201833]),
 'std_test_score': array([0.01301289]),
 'rank_test_score': array([1])}

#### Wyniki Grid i Random Search

In [51]:
print(f" Best params for SVM model by GridSearchCV :        {grid_clsf.best_params_}")
print(f" Best score with those params :                     {grid_clsf.best_score_}")
print(f" Best params for SVM model by RandomizedSearchCV :  {random_clsf.best_params_}")
print(f" Best score with those params :                     {random_clsf.best_score_}")

 Best params for SVM model by GridSearchCV :        {'C': 3, 'kernel': 'linear'}
 Best score with those params :                     0.5794428429158663
 Best params for SVM model by RandomizedSearchCV :  {'kernel': 'linear', 'C': 2}
 Best score with those params :                     0.572018329576608


### SVM model

From GridSearchCV method the best params por SVC model will be:\
C:      1\
kernel: linear

In [52]:
#kernel="linear", C=1, gamma="auto", class_weight="balanced"

In [59]:
scores = []

In [60]:
# bez wyników z grid ani random
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)

pred = svm_model.predict(X_val)

precision = metrics.precision_score(y_val, pred)
recall = metrics.recall_score(y_val, pred)
roc_auc = metrics.roc_auc_score(y_val, pred)
f1 = metrics.f1_score(y_val, pred)

scores.append({"model params": "bez eksploracji parametrów", 
                "precision": precision, 
                "recall": recall,
                "ROC AUC": roc_auc,
                "F1": f1
                })


In [61]:
# paramerty grid
svm_model_grid = svm.SVC(kernel="linear", C=3, gamma="auto", class_weight="balanced")
svm_model_grid.fit(X_train, y_train)

pred = svm_model_grid.predict(X_val)

precision1 = metrics.precision_score(y_val, pred)
recall1 = metrics.recall_score(y_val, pred)
roc_auc1 = metrics.roc_auc_score(y_val, pred)
f11 = metrics.f1_score(y_val, pred)

scores.append({"model params": "GridSearch", 
                "precision": precision1, 
                "recall": recall1,
                "ROC AUC": roc_auc1,
                "F1": f11
                })


In [62]:
# parametry ransom
svm_model_random = svm.SVC(kernel="linear", C=2, gamma="auto", class_weight="balanced")
svm_model_random.fit(X_train, y_train)

pred = svm_model_random.predict(X_val)

precision2 = metrics.precision_score(y_val, pred)
recall2 = metrics.recall_score(y_val, pred)
roc_auc2 = metrics.roc_auc_score(y_val, pred)
f12 = metrics.f1_score(y_val, pred)

scores.append({"model params": "RandomSearch", 
                "precision": precision2, 
                "recall": recall2,
                "ROC AUC": roc_auc2,
                "F1": f12
                })

                

### Wyniki 

In [63]:
best_params = pd.DataFrame(scores,columns=["model params","precision","recall","ROC AUC","F1"])

In [64]:
best_params

,model params,precision,recall,ROC AUC,F1
0,bez eksploracji parametrów,0.695341,0.518717,0.718176,0.594181
1,GridSearch,0.462774,0.847594,0.745502,0.598678
2,RandomSearch,0.462774,0.847594,0.745502,0.598678
